In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score, r2_score
import matplotlib.pyplot as plt
from math import sqrt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import keras
import pickle
import xgboost as xgb

## Loading training data

In [2]:
data = pd.read_csv("../Data/unscaled.csv")
del data['Unnamed: 0']
scaler = StandardScaler()
# data = data[:200000]
y = data['block_min']
X = data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
X_train_scaled = X_train.copy()
X_train_scaled = X_train_scaled.drop(['block_min'], axis=1)
X_test_scaled = X_test.copy()
X_test_scaled = X_test_scaled.drop(['block_min'], axis=1)
X_train_scaled[X_train_scaled.columns[:]] = scaler.fit_transform(X_train_scaled[X_train_scaled.columns[:]])
X_test_scaled[X_test_scaled.columns[:]] = scaler.fit_transform(X_test_scaled[X_test_scaled.columns[:]])

## User Input

In [3]:
def inputText():
    print('Choose the model you want to load:\n')
    print('1. Linear Regression')
    print('2. Random Forest')
    print('3. Neural Net')
    print('4. ADABoost')
    print('5. XGBoost')

def linear():
    print('Choose type of linear regressor:\n')
    print('1. Simple LR')
    print('2. Lasso LR')
    print('3. Ridge LR')
    name = ''
    mtype = int(input())
    if mtype == 1:
        name = '../weights/simpleLR.sav'
    elif mtype == 2:
        name = '../weights/lassoLR.sav'
    elif mtype == 3:
        name = '../weights/ridgeLR.sav'
    else:
        print('Invalid input, simple linear regression model will be chosen')
        name = '../weights/simpleLR.sav'
    return name

def choosemodel():
    fileName = ''
    flag = False
    inputText()
    choice = int(input())
    if choice == 1:
        fileName = linear()
    elif choice == 2:
        fileName = '../weights/randomForest.sav'
    elif choice == 3:
        fileName = '../weights/neuralNet'
        return nn(fileName)
    elif choice == 4:
        fileName = '../weights/ADABoost.sav'
    elif choice == 5:
        fileName = '../weights/xgboost.json'
        return xg(fileName)
    else:
        print('Enter 1 to 5')
        return
    loaded_model = pickle.load(open(fileName, 'rb'))
    return loaded_model

In [4]:
def nn(file):
    model = keras.models.load_model(file)
    return model

def xg(file):
    model = xgb.XGBRegressor()
    model.load_model(file)
    return model

## Loading model

In [5]:
model = choosemodel()

Choose the model you want to load:

1. Linear Regression
2. Random Forest
3. Neural Net
4. ADABoost
5. XGBoost
3


## Testing the model

In [9]:
def calculateR2(y_test,y_pred):
    v = ((y_test - y_test.mean())**2).sum()
    u = ((y_test - y_pred)**2).sum()
    return 1-u/v

def accuracy(y_pred,data):
    correct=0
    i=0
    for index,row in data.iterrows():
        if(y_pred[i]>=row['block_min'] and y_pred[i]<=row['past_max']):
            correct+=1
        i+=1
    acc = correct/data.shape[0]
    return acc 

def calculateMetric(y_pred,y_test, data):
    print("MAE=",mean_absolute_error(y_test,y_pred))
    print("RMSE=",sqrt(mean_squared_error(y_test,y_pred)))
    print("R2 Score=",r2_score(y_test,y_pred))
    print('Accuracy=', accuracy(y_pred, data))

In [10]:
y_pred = model.predict(X_test_scaled)
calculateMetric(y_pred, y_test, X_test)

MAE= 3468354118.2385836
RMSE= 5200934162.607484
R2 Score= 0.5438013654679732
Accuracy= 0.519556645639313
